<a href="https://www.kaggle.com/code/ibrahimnibrahim/fake-news-85-val-accuracy?scriptVersionId=244286373" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/FakeNewsNet.csv


In [2]:
data=pd.read_csv('/kaggle/input/fake-news/FakeNewsNet.csv')
data.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [3]:
import nltk
from nltk.corpus import stopwords

text=data['title'].apply(lambda x: str(x).split())
labels=data['real']

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

text=text.apply(lambda x : ' '.join([word for word in x if word not in stop_words]))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
text_train, text_test, labels_train, labels_test = train_test_split(text, labels, test_size=0.2, random_state=101)

In [5]:
import tensorflow as tf

vectorized_layer=tf.keras.layers.TextVectorization(ragged=True,max_tokens=10000)
vectorized_layer.adapt(text_train)

In [6]:
train_sequence=vectorized_layer(text_train)
test_sequence=vectorized_layer(text_test)

train_padded=tf.keras.utils.pad_sequences(
    train_sequence.numpy(),
    maxlen=20,
    padding='pre',
    truncating='pre',
)
test_padded=tf.keras.utils.pad_sequences(
    test_sequence.numpy(),
    maxlen=20,
    padding='pre',
    truncating='pre',
)

In [7]:
train_padded=tf.data.Dataset.from_tensor_slices(train_padded)
test_padded=tf.data.Dataset.from_tensor_slices(test_padded)

labels_train=tf.data.Dataset.from_tensor_slices(labels_train)
labels_test=tf.data.Dataset.from_tensor_slices(labels_test)

train_dataset=tf.data.Dataset.zip(train_padded,labels_train)
test_dataset=tf.data.Dataset.zip(test_padded,labels_test)

In [8]:
train_dataset=train_dataset.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE).batch(32)
test_dataset=test_dataset.cache().prefetch(tf.data.AUTOTUNE).batch(32)

In [9]:
model = tf.keras.Sequential([
   tf.keras.layers.Input(shape=(20,)),  # Input layer
   tf.keras.layers.Embedding(10000,128),  # Embedding layer
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(25, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(15, return_sequences=True)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 18, 128)             │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 18, 50)              │          30,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 18, 30)              │           7,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 540)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         138,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,506,753 (5.75 MB)

 Trainable params: 1,506,753 (5.75 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
es=tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    restore_best_weights=True,
)

In [12]:
history = model.fit(train_dataset, epochs=30, validation_data=test_dataset,callbacks=[es,])

Epoch 1/30
580/580 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.8039 - loss: 0.4611 - val_accuracy: 0.8353 - val_loss: 0.3798
Epoch 2/30
580/580 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8936 - loss: 0.2624 - val_accuracy: 0.8259 - val_loss: 0.4230
Epoch 3/30
580/580 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9441 - loss: 0.1499 - val_accuracy: 0.7955 - val_loss: 0.7416
Epoch 4/30
580/580 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9680 - loss: 0.0905 - val_accuracy: 0.8093 - val_loss: 0.9283
Epoch 5/30
580/580 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9787 - loss: 0.0587 - val_accuracy: 0.8034 - val_loss: 1.2847
Epoch 6/30
580/580 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9852 - loss: 0.0417 - val_accuracy: 0.8060 - val_loss: 1.0098
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.
